# Building a database for crime reports 

In this project, the aim is to build a database for storing data related to crimes that have occured in Boston. 

The key Postgres skills in this project will be:
* Creating a database and managing database roles
* Creating a database schema and tables with proper datatypes
* Loading data from csv files into database tables 

In [1]:
import psycopg2

# Connecting to the database and creating a new database called crime_db 
conn = psycopg2.connect(dbname = 'dq', user = 'dq')
cur = conn.cursor()
conn.autocommit = True
cur.execute("CREATE DATABASE crime_db;")
conn.autocommit = False 
conn.close()



In [2]:
# Connecting to the new crime_db database and creating a new schema
import psycopg2
conn = psycopg2.connect(dbname = 'crime_db', user = 'dq')
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes;")

In [3]:
# Commiting the changes made
conn.commit()

### Obtaining the Column Names and Sample
Now we have a database and a scheme, we can start creating some tables. Before we can create tables, we need to have a look at a sample of the data to be able to select the correct datatypes. 

In [4]:
# Reading in the csv file
import csv 
with open('boston.csv') as file:
    reader = csv.reader(file)
    reader = list(reader)
    col_headers = reader[0]
    first_row = reader[1]

In [5]:
# Printing the header and the first row 
print('Header data: ', col_headers)
print('\n')
print('First row of data: ', first_row)

Header data:  ['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


First row of data:  ['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


### Creating an Auxiliary Function 
Before we create the table to store crime data, we need to identify the correct datatypes for the columns. To help us do this, we can create a helper function, which will compute a Python set with distinct values of a column, given the file name and column index. 

This function will help us to:
* Check if an enumerated datatype could be used to represent a column 
* Compute the maximum length of any text in a column to enable the correct selection of size for VARCHAR to save space

In [6]:
# Creating a helper function to find the number of different values of each column
def get_col_set(csv_filename, col_index):
    import csv
    values = set()
    with open(csv_filename, 'r') as file:
        next(file)
        reader = csv.reader(file)
        for row in reader:
            values.add(row[col_index])
    return values

for row in range(len(col_headers)):
    lengths = get_col_set('boston.csv', row)
    print(col_headers[row], len(lengths), sep='\t')

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


### Finding the maximum length
It would be useful to know the longest word in any column which contains text data. The two columns that contain text data are 'description' and 'day_of_the_week'. 'day_of_the_week' has seven values, so it's easy to compute the max length. 

In [7]:
# printing value of col_headers to see what index the description column has
print(col_headers)
print('\n')

# using index to compute the length of the max description 
desc_colum = get_col_set('boston.csv', 2)
max_length = 0
for row in desc_colum:
    max_length = max(max_length, len(row))

print("Maximum length of the description column is: ", max_length)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


Maximum length of the description column is:  58


### Creating the table
Now we are ready to create the Boston Crimes table inside of the crimes schema. The day_of_the_week column is suitable for enumerated datatype as it only has 7 unique values. 

In [8]:
# Creating an enumerated datatype 
cur.execute("CREATE TYPE days_enum AS ENUM('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');")
conn.commit()

In [9]:
# Finding suitable column names for the table
print(col_headers)
print('\n')

# Reminding ourselves of the data contained in the file to help determine the correct datatype
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


The table column names will be the same as the column headers of the csv file. We will write out date as offence_date and latitude and longitude in full for ease of readability. The columns will have the following datatypes based on findings of unique values and max lengths:

* incident_number = integer. This will also be the Primary Key for the table
* offence_code = smallint
* description = varchar(100)
* offence_date = date 
* day_of_week = days_enum 
* latitude = decimal(10,8)
* longitude = decimal(11,8)

In [14]:
table_query = '''
        CREATE TABLE crimes.boston_crimes (
        incident_number integer PRIMARY KEY, 
        offence_code smallint,
        description varchar(100),
        offence_date date,
        day_of_week days_enum,
        latitude decimal(10,8),
        longitude decimal(11,8));
'''
cur.execute(table_query)


In [15]:
# Commiting the changes
conn.commit()

### Loading the data
Now that we have the boston_crimes table, we can load data into it. In this project, we will use the cursor.copy_expert() method. 

In [22]:
# Loading the data from the csv file into the new postgres table
with open('boston.csv', 'r') as file:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER", file)

In [25]:
# Viewing if data loaded correctly     
cur.execute("SELECT * FROM crimes.boston_crimes LIMIT 10;")
data = cur.fetchall()

print(data, sep = '\n')

[(1, 619, 'LARCENY ALL OTHERS', datetime.date(2018, 9, 2), 'Sunday', Decimal('42.35779134'), Decimal('-71.13937053')), (2, 1402, 'VANDALISM', datetime.date(2018, 8, 21), 'Tuesday', Decimal('42.30682138'), Decimal('-71.06030035')), (3, 3410, 'TOWED MOTOR VEHICLE', datetime.date(2018, 9, 3), 'Monday', Decimal('42.34658879'), Decimal('-71.07242943')), (4, 3114, 'INVESTIGATE PROPERTY', datetime.date(2018, 9, 3), 'Monday', Decimal('42.33418175'), Decimal('-71.07866441')), (5, 3114, 'INVESTIGATE PROPERTY', datetime.date(2018, 9, 3), 'Monday', Decimal('42.27536542'), Decimal('-71.09036101')), (6, 3820, 'M/V ACCIDENT INVOLVING PEDESTRIAN - INJURY', datetime.date(2018, 9, 3), 'Monday', Decimal('42.29019621'), Decimal('-71.07159012')), (7, 724, 'AUTO THEFT', datetime.date(2018, 9, 3), 'Monday', Decimal('42.30607218'), Decimal('-71.08273260')), (8, 3301, 'VERBAL DISPUTE', datetime.date(2018, 9, 3), 'Monday', Decimal('42.32701648'), Decimal('-71.10555088')), (9, 301, 'ROBBERY - STREET', datetime.d

### Revoking Public Privileges
Now that we have a database and a scheme, we need to create two groups and follow the least privilege principle. 

In [26]:
# revoking all the privileges of the public group on the public schema

cur.execute("REVOKE ALL ON SCHEMA public FROM public;")

# revoking all privileges of the public group on the crime_db database
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

conn.commit()

### Creating User Groups 
In the previous step, we made sure that the new groups don't inadvertently inherit privileges from the public group. Now we can create the two new groups; readonly and readwrite. 

In [33]:
# Creating new user groups
cur.execute("CREATE GROUP readonly NOLOGIN;")

cur.execute("CREATE GROUP readwrite NOLOGIN;")

# Granting connect and usage privileges to the users
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")

# Granting specific privileges to each group on all tables
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

conn.commit()

### Creating Users 
Now that we have groups which have the correct privileges, we can create specific users. We will create two users, one for each group. 

In [36]:
# creating data_analyst user
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")

# creating data_scientist user
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")

conn.commit()

### Testing 
The Postgres database is now set up! We can write some queries to perform test and ensure that the database has been set up correctly. 

In [42]:
# Querying the internal pg_roles table
cur.execute("SELECT * FROM pg_roles;")
result = cur.fetchall()
print(result)
print('\n')

# Querying the internal information_schema.table_provileges table

cur.execute("SELECT grantee, privilege_type FROM information_schema.table_privileges WHERE grantee = 'readonly' OR grantee = 'readwrite';")
result2 = cur.fetchall()
print(result2)

[('dq', True, True, True, True, True, True, True, -1, '********', None, None, 10), ('readonly', False, True, False, False, False, False, False, -1, '********', None, None, 16415), ('readwrite', False, True, False, False, False, False, False, -1, '********', None, None, 16416), ('data_analyst', False, True, False, False, False, True, False, -1, '********', None, None, 16417), ('data_scientist', False, True, False, False, False, True, False, -1, '********', None, None, 16418)]


[('readonly', 'SELECT'), ('readwrite', 'INSERT'), ('readwrite', 'SELECT'), ('readwrite', 'UPDATE'), ('readwrite', 'DELETE')]
